<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro/References" data-toc-modified-id="Intro/References-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro/References</a></span></li><li><span><a href="#Scaling-not-clipping?" data-toc-modified-id="Scaling-not-clipping?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scaling not clipping?</a></span></li><li><span><a href="#Pytorch" data-toc-modified-id="Pytorch-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Pytorch</a></span></li><li><span><a href="#Pytorch-monitor" data-toc-modified-id="Pytorch-monitor-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Pytorch monitor</a></span><ul class="toc-item"><li><span><a href="#Vectorizing-is-terrible/awesome." data-toc-modified-id="Vectorizing-is-terrible/awesome.-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Vectorizing is terrible/awesome.</a></span></li></ul></li><li><span><a href="#Works-beautifully-for-CartPole" data-toc-modified-id="Works-beautifully-for-CartPole-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Works beautifully for CartPole</a></span></li><li><span><a href="#Debugging" data-toc-modified-id="Debugging-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Debugging</a></span><ul class="toc-item"><li><span><a href="#Things-I-tried" data-toc-modified-id="Things-I-tried-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Things I tried</a></span></li></ul></li><li><span><a href="#Lunar-Lander" data-toc-modified-id="Lunar-Lander-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Lunar Lander</a></span></li></ul></div>

# Intro/References

A Distributional Perspective on Reinforcement Learning
https://deepmind.com/blog/going-beyond-average-reinforcement-learning/
https://arxiv.org/abs/1707.06887

Distributional Reinforcement Learning with Quantile Regression
https://arxiv.org/abs/1710.10044

Distributional RL
https://mtomassoli.github.io/2017/12/08/distributional_rl/

An Analysis of Categorical Distributional Reinforcement Learning
https://arxiv.org/abs/1802.08163



    
# Scaling not clipping?

# Pytorch
# Pytorch monitor

In [2]:
from collections import (deque, defaultdict)
from functools import partial
import math
import random
import sys

import attr
import gym
from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
from pytorch_monitor import init_experiment, monitor_module
from running_stats import RunningStats
from smooth import smooth  # timeseries smoothing function
import torch
from torch import nn
import torch.nn.functional as F


cartpole = gym.make('CartPole-v1')
lunarlander = gym.make('LunarLander-v2')
plt.style.use('seaborn-white')

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'  

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
@attr.s
class Memory(deque):
    """ Experience Replay Memory class. """
    size = attr.ib()
    minibatch_size = attr.ib()

    def append(self, thing):
        if len(self) > self.size - 1:
            self.popleft()
        return super().append(thing)

    def sample(self):
        batch_size = min(len(self), self.minibatch_size)
        data = random.sample(self, batch_size)
        states = torch.stack([record[0] for record in data])
        actions = torch.tensor([record[1] for record in data], dtype=torch.long)
        rewards = torch.tensor([record[2] for record in data], dtype=torch.float)
        states_ = torch.stack([record[3] for record in data])
        dones = torch.tensor([record[4] for record in data], dtype=torch.long)
        return (states, actions, rewards, states_, dones)

    
class ValueDistribution(torch.nn.Module):
    def __init__(self, state_shape, action_shape, vmin, vmax, num_atoms=51, num_hidden1_units=64):
        super().__init__()
        self.state_shape = state_shape
        self.action_shape = action_shape
        self.vmin = vmin
        self.vmax = vmax
        self.num_atoms = num_atoms
        self.atoms = torch.linspace(self.vmin, self.vmax, self.num_atoms)
        self.linear1 = nn.Linear(self.state_shape, num_hidden1_units)
        self.linear2 = nn.Linear(num_hidden1_units, self.action_shape * self.num_atoms)
                
    def forward(self, x):
        """ Return (actions x atoms). """
        x1 = F.leaky_relu(self.linear1(x))
        x2 = self.linear2(x1).reshape(-1, self.action_shape, self.num_atoms)
        out = F.softmax(x2, dim=2)  # (actions x atoms)
        if x.dim() == 1:
            batch_size = 1
        else:
            batch_size = x.size(0)
        assert out.size() == torch.Size((batch_size, self.action_shape, self.num_atoms))
        if hasattr(self, 'monitor'):
            self.monitor('x1', x1, track_data=False, track_grad=True)
            self.monitor('x2', x2, track_data=False, track_grad=True)
            self.monitor('out', out, track_data=False, track_grad=True)
        return out
    
    def predict_action_values(self, states):
        """ Return (batch-size x actions). """
        distribution = self.forward(states)
        weighted_distribution = distribution * self.atoms
        out = weighted_distribution.sum(dim=2).squeeze()  # (batch-size x actions)
        dims = states.dim()
        assert out.size() == torch.Size((self.action_shape,))
        return out
        
    def get_action(self, state):        
        values = self.predict_action_values(state)
        action = values.argmax()
        return action        

## Vectorizing is terrible/awesome.

This is the algo we'll be implementing. 

![The C51 algorithm](assets/C51-algo.png)

Vectorizing this code is *very* important. Even on my lowely macbook, the fully vectorized version of this algorithm that accepts a minibatch runs about *30 times* faster than a naive implementation that's called inside a loop. This cashes out to 1000 training episodes of LunarLander in about 10 minutes, verses five hours. 

My process for vectoring this code was.. to sort of squint at it. 
Seriously. I wasn't even sure if I should first generalize it to accept minibatch tensors and then remove the loop, or vice versa. 

Squinting at it, thought, (and stepping through it line by line in Jupyter a few times), it became clear that the would actually tricky to vectorize: 

![The bastard lines]](assets/C51-algo-large.png)

I decided to take the easy wins first, and first converted the `categorical_loss` function to accept minibatches first. This is straighforward, mostly just reshaping and expanding tensors. Pytorch's `squeeze` and `unsqueeze` methods have fun names and are great for this. 

Those two lines, though, were bloody horrible. 

They ended up cashing out into the following dense six lines of python:

```python
offset_bound = target_net.num_atoms * batch_size - target_net.num_atoms
idx_offset = torch.range(0, offset_bound, target_net.num_atoms).unsqueeze(1).expand_as(m)
lo_idx = (lo + idx_offset).view(-1).type(torch.long)
hi_idx = (hi + idx_offset).view(-1).type(torch.long)
lo_component = m.view(-1).index_add(0, lo_idx, (probabilities * (hi - b_j)).view(-1) )
hi_component = m.view(-1).index_add(0, hi_idx, (probabilities * (b_j - lo)).view(-1) )
m += lo_component.resize_as(m) + hi_component.resize_as(m)       
```

The main insight is that the `lo` and `hi` tensors contain routing information. They tend look like this:

```
lo:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 10],
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10]])
hi:
tensor([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [3, 4, 5, 6, 7, 7, 8, 9, 10, 10, 10],
        [2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10]])
```

This is for a three-transition test minibatch. The values are the target indicies for where probability needs to accumulate inside `m` our new probability-mass tensor.

Eventually after squinting a lot at the PyTorch docs, I figured out that PyTorch's [`index_add`](https://pytorch.org/docs/stable/tensors.html?highlight=index_add#torch.Tensor.index_add_) method would do the trick. 

Usings `index_add` requires that all the tensors be unrolled, which is why we need index-offsets. Put it together, and you're done.



In [4]:
def categorical_vectorized_loss(online_net, target_net, transitions, discount): 
    states, actions, rewards, states_, dones = transitions
    not_dones = (1 - dones).type(torch.FloatTensor)
    atoms = target_net.atoms
    probabilities = target_net.forward(states_)
    Q_x_ = (probabilities * atoms).sum(2)
    batch_size = states.shape[0]
    assert Q_x_.shape == torch.Size((batch_size, target_net.action_shape)), f'Got: {Q_x_.shape}, expected: {(batch_size, target_net.action_shape)}'
    a_star = Q_x_.argmax(dim=1) 
    assert a_star.shape == torch.Size((batch_size,)), f'Got {a_star.shape}, expected: ((batch_size,))'
    
    # compute the projected probability:
    delta_z = (target_net.vmax - target_net.vmin)/(target_net.num_atoms - 1)    
    # select only the probabilities distributions for the a_star actions:
    probabilities = probabilities[range(batch_size), a_star]
    T_zj = rewards.unsqueeze(1) + discount * atoms * not_dones.unsqueeze(1)
    b_j = (T_zj.clamp(target_net.vmin, target_net.vmax) - target_net.vmin) / delta_z  # correct    
    lo = b_j.floor()        
    hi = b_j.ceil()
    m = torch.zeros(batch_size, target_net.num_atoms, dtype=torch.float)
    lo_component = torch.zeros_like(m.view(-1))
    hi_component = torch.zeros_like(m.view(-1))
    # offset will be used for indexing when we flatten the tensors into vectors:
    offset_bound = target_net.num_atoms * batch_size - target_net.num_atoms
    idx_offset = torch.range(0, offset_bound, target_net.num_atoms).unsqueeze(1).expand_as(m)
    lo_idx = (lo + idx_offset).view(-1).type(torch.long)
    hi_idx = (hi + idx_offset).view(-1).type(torch.long)
    lo_component = m.view(-1).index_add(0, lo_idx, (probabilities * (hi - b_j)).view(-1) )
    hi_component = m.view(-1).index_add(0, hi_idx, (probabilities * (b_j - lo)).view(-1) )
    m += lo_component.reshape(batch_size, target_net.num_atoms) + hi_component.reshape(batch_size, target_net.num_atoms)
    # cross enthropy is Sigma <true> log <unnatural>, so for us is: target log(online)
    online_distribution = online_net.forward(states)[range(batch_size), actions]
    return -( m * online_distribution.log() ).sum(1).mean()

In [5]:
@attr.s
class CategoricalAgent:
    env = attr.ib()
    discount = attr.ib(default=0.99)
    epsilon_max = attr.ib(default=1.0)
    epsilon_min = attr.ib(default=0.01)
    annealing_const = attr.ib(default=.001)  # aka Lambda
    minibatch_size = attr.ib(default=32)
    memory_size = attr.ib(default=int(1e6))
    num_episodes = attr.ib(default=1000)  # num of episodes in a training epoch
    render_every = attr.ib(default=20)  # set to zero to turn off rendering
    update_target_every = attr.ib(default=200)
    vmin = attr.ib(default=-10)
    vmax = attr.ib(default=10)
    num_atoms = attr.ib(default=51)
    learning_rate = attr.ib(default=0.000001)
    monitor_total = attr.ib(default=10)
    logger = attr.ib(default=None)
    use_kaiming_init = attr.ib(default=True)
    weight_decay = attr.ib(default=0)
    use_lr_scheduler = attr.ib(default=True)
    max_gradient_norm = attr.ib(default=1)
    learning_rate_floor = attr.ib(default=2e-06)
    learning_rate_patience = attr.ib(default=50)
    learning_rate_cooldown = attr.ib(default=50)
    learning_rate_smoothing_window = attr.ib(default=20)
    
    def __attrs_post_init__(self):
        self.steps = 0
        state_shape = self.env.observation_space.shape[0]
        self.memory = Memory(self.memory_size, self.minibatch_size)
        self.action_shape = self.env.action_space.n
        self.online_net = ValueDistribution(state_shape=state_shape, action_shape=self.action_shape, vmin=self.vmin, vmax=self.vmax, num_atoms=self.num_atoms)
        self.target_net = ValueDistribution(state_shape=state_shape, action_shape=self.action_shape, vmin=self.vmin, vmax=self.vmax, num_atoms=self.num_atoms)
        self.init_layers()
        self.target_net.load_state_dict(self.online_net.state_dict())
        self.optimizer = torch.optim.Adam(self.online_net.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        self.steps = 0
        self.target_net_q_values = deque([], 1000)
        self.episode_rewards = deque([], 1000)
        self.training_loss = deque([], 1000)
        self.layer1_grad_ratio = deque([], 1000)
        self.layer2_grad_ratio = deque([]  , 1000)
        self.monitor_every = self.num_episodes//self.monitor_total
        self.init_learning_rate_scheduler()
        self.reward_normalizer = RunningStats((),)
        self.filter_reward_buffer = 0
    
    def init_layers(self):
        if not self.use_kaiming_init:
            return                
        for param in self.online_net.parameters():
            if param.dim() < 2:
                continue
            leaky_relu_value = 0.01
            nn.init.kaiming_normal_(param, a=leaky_relu_value)
        
    
    def init_learning_rate_scheduler(self):
        if not self.use_lr_scheduler:
            return        
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, 
            'max', 
            factor=.5, 
            patience=self.learning_rate_patience,
            cooldown=self.learning_rate_cooldown,
            min_lr=self.learning_rate_floor,
            verbose=True
        )

    def step_learning_rate(self, episode):
        window_len = 11
        if self.use_lr_scheduler and episode > window_len:    
            rewards = np.array(self.episode_rewards[-self.learning_rate_smoothing_window:])
            smoothed_reward = smooth(rewards), window_len=window_len).mean()
            self.scheduler.step(smoothed_reward)
                
    def render(self, episode):
        if self.render_every and episode % self.render_every == 0:
            self.env.render()

    def training_progress_report(self, episode):
        last_ep = self.episode_rewards[-1]
        ten_ep_mean = sum(self.episode_rewards[-10:])/len(self.episode_rewards[-10:])
        hundred_ep_mean = sum(self.episode_rewards[-100:])/len(self.episode_rewards[-100:])
        try:
            lin1_grad = self.online_net.linear1.weight.grad.norm()
            lin2_grad = self.online_net.linear2.weight.grad.norm()
        except:
            lin1_grad =-666
            lin2_grad = -666
        return f'Ep: {episode} // steps: {self.steps} // last ep reward: {last_ep:.2f} // {min(10, len(self.episode_rewards[-10:]))}-ep mean: {ten_ep_mean:.2f} // {min(100, len(self.episode_rewards[-100:]))}-ep mean: {hundred_ep_mean:.2f}, layer1 grad: {lin1_grad:.2f}, layer2 grad: {lin2_grad:.2f}'

    def monitor(self):
        if not hasattr(self.target_net, 'monitoring'):
            return
        if self.monitor_every and self.steps % self.monitor_every == 0:
            self.target_net.monitoring(True)
        else:
            self.target_net.monitoring(False)
            
    def log_params(self, episode):
        if not self.logger:
            return     
        if episode % 5 != 0:
            return 
        self.logger.add_scalar('train loss', self.training_loss[-1], episode)
        self.logger.add_scalar('episode reward', self.episode_rewards[-1], episode)  
        self.logger.add_scalar('L1 size ratio', self.layer1_grad_ratio[-1], episode)
        self.logger.add_scalar('L2 size ratio', self.layer2_grad_ratio[-1], episode)        
        for idx, param in enumerate(self.target_net.parameters()):
            if param.dim() == 1:
                continue
            if not hasattr(param, 'grad'):
                continue
            self.logger.add_scalar(f'layer {idx//2 + 1} gradient', param.grad.norm(), episode)            

    def filter_reward(self, reward):
        reward = np.clip(reward, -1, 1)
        return reward

    def replay(self):
        batch = self.memory.sample()
        loss = categorical_vectorized_loss(self.online_net, self.target_net, batch, self.discount)
        self.optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(self.online_net.parameters(), self.max_gradient_norm)
        self.optimizer.step()
        return loss.item()/self.minibatch_size

    def train(self, start=0):
        for episode in range(start, start + self.num_episodes):
            episode_done = False
            episode_reward = 0
            episode_loss = 0
            state = torch.tensor(self.env.reset(), dtype=torch.float)
            self.target_net_q_values.append(self.target_net.predict_action_values(state).max().item())
            if self.logger:
                if self.steps == 0:
                    self.logger.add_graph(self.target_net, state)            
                self.logger.add_scalar('Target net Q values', self.target_net_q_values[-1], episode)                

            layer1_size = self.target_net.linear1.weight.norm().item()
            layer2_size = self.target_net.linear2.weight.norm().item()                
            while not episode_done:
                self.monitor()
                epsilon = self.epsilon_min + (self.epsilon_max - self.epsilon_min) * math.exp(-self.annealing_const * self.steps)
                self.steps += 1                
                if random.random() < epsilon:
                    action = random.randint(0, self.action_shape-1)
                else:
                    action = self.online_net.get_action(state).item()
                self.render(episode)
                self.monitor()
                state_, reward, episode_done, _ = self.env.step(action)
                state_ = torch.tensor(state_, dtype=torch.float)
                reward = self.filter_reward(reward)
                episode_reward += reward
                self.memory.append((state, action, reward, state_, episode_done))
                state = state_
                if self.steps < 2:
                    continue
                episode_loss += self.replay()
                
                if self.steps % self.update_target_every == 0:
                    self.target_net.load_state_dict(self.online_net.state_dict())
                if episode_done:
                    self.episode_rewards.append(episode_reward)
                    self.training_loss.append(episode_loss)
                    print(self.training_progress_report(episode), end='\r', flush=True)

                    layer1_ratio = self.target_net.linear1.weight.norm().item() / (layer1_size + 1e-5)
                    layer2_ratio = self.target_net.linear2.weight.norm().item() / (layer2_size + 1e-5)
                    self.layer1_grad_ratio.append(layer1_ratio)
                    self.layer2_grad_ratio.append(layer2_ratio)
                                        
                    self.log_params(episode)
                    self.step_learning_rate(episode)
        
            

SyntaxError: invalid syntax (<ipython-input-5-16dedb5778c4>, line 76)

# Works beautifully for CartPole

In [1]:
for idx in range(2):
    lr = 0.001
    weight_decay = 0.0001
    title = f'Run {idx}: LR: {lr:.3f} / L2 weight_decay: {weight_decay:.3f}'
    config = {
        'title':title,
        'log_dir':'tensorboard-data/tuning-categorical',
        'random_seed':idx
    }
    logger, config = init_experiment(config)
    print(title)        
    print(config['log_dir'])
    agent = CategoricalAgent(
        cartpole, 
        learning_rate=lr, 
        weight_decay=weight_decay, 
        use_lr_scheduler=True, 
        logger=logger,
        num_episodes=1500,
        max_gradient_norm=.5,
        monitor_total=5,
        use_kaiming_init=True,
        minibatch_size=32
        
    )
    monitor_module(
        agent.target_net, 
        logger, 
        track_data=False,
        track_grad=True,
        track_update=True,
        track_update_ratio=True
    )
    # agent.train()

NameError: name 'init_experiment' is not defined

# Debugging

However, it's quite unstable in the LunarLander environment. I'm still not sure why -- it may be bugs, it may be hyper-parameters. 

![Q values](assets/q values.png)
![Episode rewards](assets/episode rewards.png)

These images show what three typical runs (Different number of epochs, but otherwise similar hyperparameters): the agent's performance is hugely variable, sometimes doing quite well, sometimes doing terribly, with the (heavily) smoothed trend line more or less flat. The Q values seem to converge, but also have high variance. 

The gradients through the ReLu/LeakyReLu layers tend to be *ridiculously* large. I'm not convinced this is the source of the performance problem in LunarLander, as L2 and gradient-norm clipping are successful at keeping the norm of the weights constant during training, and learning is stable with similarly large gradients in CartPole. 

I've spent over a week attempting to debug the LunarLander performance problems -- for now I'm putting this project on the back-burner. 


## Things I tried

Singlely and in combinations:

- Simple three-armed bandit test to confirm that the agent can learn arbitrary distributions
- Careful step-through and analysis of my implementation of the Categorical Algorithm, plus comparing against the output of other people's Categorical Algorithm implementations.
- ReLU, PreReLU and LeakyReLU activation functions
- No layer initialization, Xavier Normal, Xavier Uniform and Kaiming Normal init
- Batch norm layers -- I later found a reference in the [Weight Normalization](https://arxiv.org/abs/1602.07868) paper suggesting that "the noise introduced by estimating the minibatch statistics destabilizes the [DQN] learning process".
- Weight normalization
- Reward clipping (either to [-1, 1], or arbitrary values that seemed reasonable given the histogram of rewards, to to [Vmin, Vmax]).
- Exponential reward smoothing and normalization
- Different layer architetures
- Using a learning rate annealing schedule with several differ hyperparameter settings
- Random hyperparameter searches 
- Probably other things I forgot to include in my log file


Additional things I'd *like* to do at some point:
- track the magnitude of the policy changes during training. If the policy is changing too much, it probably suggests some kind of learning instability
- Visualizing the reward distribution for specific states -- even if this doesn't help with debugging, it'll be **cool**
- Bodging in DoubleDQN -- perhaps the problem is simply that the Categorical agent is too optimistic? 
- Talking with more people about this. :)

# Lunar Lander

In [50]:
for idx in range(2):
    lr = 0.001
    weight_decay = 0.0001
    title = f'Run {idx}: LR: {lr:.3f} / L2 weight_decay: {weight_decay:.3f}'
    config = {
        'title':title,
        'log_dir':'tensorboard-data/tuning-categorical',
        'random_seed':idx
    }
    logger, config = init_experiment(config)
    print(title)        
    print(config['log_dir'])
    agent = CategoricalAgent(
        lunarlander, 
        learning_rate=lr, 
        weight_decay=weight_decay, 
        use_lr_scheduler=True, 
        logger=logger,
        num_episodes=4500,
        max_gradient_norm=.5,
        monitor_total=5,
        use_kaiming_init=True,
        minibatch_size=32,
        learning_rate_floor=7e-6,
        learning_rate_patience=150,
        learning_rate_cooldown=150,
        update_target_every=500,
        learning_rate_smoothing_window=50
    )
    monitor_module(
        agent.target_net, 
        logger, 
        track_data=False,
        track_grad=True,
        track_update=True,
        track_update_ratio=True
    )
    agent.train()
    


Run 0: LR: 0.001 / L2 weight_decay: 0.000
tensorboard-data/tuning-categorical


/Users/liavkoren/Envs/ai-gym/lib/python3.6/site-packages/torch/onnx/utils.py:365: UserWarning: ONNX export failed on ATen operator reshape because torch.onnx.symbolic.reshape does not exist
  .format(op_name, op_name))


Epoch   257: reducing learning rate of group 0 to 5.0000e-04.an: -32.08 // 100-ep mean: -26.76, layer1 grad: 0.02, layer2 grad: 0.052
Epoch   558: reducing learning rate of group 0 to 2.5000e-04.ean: -38.05 // 100-ep mean: -24.71, layer1 grad: 0.03, layer2 grad: 0.08
Epoch   859: reducing learning rate of group 0 to 1.2500e-04.ean: -25.02 // 100-ep mean: -28.37, layer1 grad: 0.02, layer2 grad: 0.037
Epoch  1160: reducing learning rate of group 0 to 6.2500e-05.mean: -41.12 // 100-ep mean: -27.48, layer1 grad: 0.02, layer2 grad: 0.03
Epoch  1461: reducing learning rate of group 0 to 3.1250e-05.mean: -16.30 // 100-ep mean: -32.09, layer1 grad: 0.01, layer2 grad: 0.025
Epoch  1762: reducing learning rate of group 0 to 1.5625e-05.mean: -28.45 // 100-ep mean: -29.21, layer1 grad: 0.02, layer2 grad: 0.04
Epoch  2063: reducing learning rate of group 0 to 7.8125e-06.mean: -26.11 // 100-ep mean: -29.50, layer1 grad: 0.02, layer2 grad: 0.04
Epoch  2364: reducing learning rate of group 0 to 7.0000

KeyboardInterrupt: 